In [204]:
! pip install yfinance
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [231]:
import yfinance as yf
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd

In [246]:
data = yf.download("^STOXX50E", period="max")

[*********************100%%**********************]  1 of 1 completed


In [247]:
data = data.reset_index(drop=False)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-03-30,4177.669922,4207.759766,4160.350098,4181.029785,4181.029785,0
1,2007-04-02,4177.310059,4191.759766,4163.770020,4189.549805,4189.549805,0
2,2007-04-03,4199.540039,4248.069824,4199.540039,4246.299805,4246.299805,0
3,2007-04-04,4252.709961,4261.830078,4242.049805,4261.830078,4261.830078,0
4,2007-04-05,4259.979980,4274.169922,4253.790039,4271.540039,4271.540039,0


In [248]:
data = data.drop('Adj Close', axis=1)
data.head()

,Date,Open,High,Low,Close,Volume
0,2007-03-30,4177.669922,4207.759766,4160.350098,4181.029785,0
1,2007-04-02,4177.310059,4191.759766,4163.770020,4189.549805,0
2,2007-04-03,4199.540039,4248.069824,4199.540039,4246.299805,0
3,2007-04-04,4252.709961,4261.830078,4242.049805,4261.830078,0
4,2007-04-05,4259.979980,4274.169922,4253.790039,4271.540039,0


In [184]:
data.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [249]:
X = data.drop('Close', axis=1)
y = data[['Date', 'Close']]
y = y[4:].drop('Date', axis=1).reset_index(drop=True).values.reshape(-1, 1, 1)
X, y

(           Date         Open         High          Low    Volume
 0    2007-03-30  4177.669922  4207.759766  4160.350098         0
 1    2007-04-02  4177.310059  4191.759766  4163.770020         0
 2    2007-04-03  4199.540039  4248.069824  4199.540039         0
 3    2007-04-04  4252.709961  4261.830078  4242.049805         0
 4    2007-04-05  4259.979980  4274.169922  4253.790039         0
 ...         ...          ...          ...          ...       ...
 4291 2024-05-13  5083.540039  5087.919922  5068.060059  23625800
 4292 2024-05-14  5079.370117  5083.549805  5057.939941  35579700
 4293 2024-05-15  5083.250000  5102.299805  5075.060059  28208300
 4294 2024-05-16  5094.770020  5101.410156  5068.109863  26348900
 4295 2024-05-17  5061.439941  5068.390137  5040.229980  33517100
 
 [4296 rows x 5 columns],
 array([[[4271.54003906]],
 
        [[4301.47021484]],
 
        [[4293.20019531]],
 
        ...,
 
        [[5100.89990234]],
 
        [[5072.45019531]],
 
        [[5064.14013

In [250]:
window_size = 5

# Initialize an empty list to store 3D arrays
X_3d_list = []

# Iterate through each window
for i in range(len(X) - window_size + 1):
    # Extract the data for the current window
    window_data = X.iloc[i:i+window_size]
    # Convert the window data to a 2D numpy array
    window_array = window_data.drop(columns=['Date']).to_numpy()
    # Append the 2D array with the date column included as the first feature
    window_array_with_time = np.insert(window_array, 0, window_data['Date'], axis=1)
    # Append the 2D array to the list
    X_3d_list.append(window_array_with_time)

# Stack the list of 3D arrays into a single 3D numpy array
X_3d = np.stack(X_3d_list)


X_3d[1], X_3d[-1]

(array([[1.17547200e+18, 4.17731006e+03, 4.19175977e+03, 4.16377002e+03,
         0.00000000e+00],
        [1.17555840e+18, 4.19954004e+03, 4.24806982e+03, 4.19954004e+03,
         0.00000000e+00],
        [1.17564480e+18, 4.25270996e+03, 4.26183008e+03, 4.24204980e+03,
         0.00000000e+00],
        [1.17573120e+18, 4.25997998e+03, 4.27416992e+03, 4.25379004e+03,
         0.00000000e+00],
        [1.17616320e+18, 4.27568018e+03, 4.30766016e+03, 4.26700977e+03,
         0.00000000e+00]]),
 array([[1.71555840e+18, 5.08354004e+03, 5.08791992e+03, 5.06806006e+03,
         2.36258000e+07],
        [1.71564480e+18, 5.07937012e+03, 5.08354980e+03, 5.05793994e+03,
         3.55797000e+07],
        [1.71573120e+18, 5.08325000e+03, 5.10229980e+03, 5.07506006e+03,
         2.82083000e+07],
        [1.71581760e+18, 5.09477002e+03, 5.10141016e+03, 5.06810986e+03,
         2.63489000e+07],
        [1.71590400e+18, 5.06143994e+03, 5.06839014e+03, 5.04022998e+03,
         3.35171000e+07]]))

In [251]:
X_train, X_remainder, y_train, y_remainder = train_test_split(X_3d, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_remainder, y_remainder, test_size=0.333, shuffle=False)
X_test, X_forecast, y_test, y_forecast = train_test_split(X_test, y_test, test_size=0.2, shuffle=False)

In [216]:
len(X_train), len(X_val), len(X_test), len(X_forecast), len(y_val)

(3004, 859, 343, 86, 859)

In [252]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [218]:
batch_size = 32

In [219]:
train_dataset = CustomDataset(X_train, y_train)
validation_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)
forecast_dataset = CustomDataset(X_forecast, y_forecast)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
forecast_dataloader = DataLoader(forecast_dataset, batch_size=batch_size, shuffle=False)

In [220]:
for batch in train_dataloader:
    pass

In [221]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        self.layer_dim = num_layers
        self.hidden_dim = hidden_dim

        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [222]:
input_dim = len(X_train[0])
output_dim = 1

In [223]:
num_epochs = 10
num_trials = 10

In [228]:
# Define the objective function for hyperparameter optimization
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    hidden_dim = trial.suggest_int('hidden_dim', 32, 512, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    # Define and initialize the model
    model = GRUModel(input_dim, hidden_dim, num_layers, output_dim, dropout)

    # Define optimizer and criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_dataloader:
            optimizer.zero_grad()
            inputs = inputs.float()
            targets = targets.float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        validation_loss = 0
        with torch.no_grad():
            for inputs, targets in validation_dataloader:
                inputs = inputs.float()
                targets = targets.float()
                outputs = model(inputs)
                validation_loss += criterion(outputs, targets).item()

        validation_loss /= len(validation_dataloader)

        # Report intermediate results to Optuna
        trial.report(validation_loss, epoch)

        # Handle pruning based on the intermediate results
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return validation_loss

# Create a study object with TPE sampler
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=num_trials)

# Get the best hyperparameters
best_params = study.best_params




[I 2024-05-18 20:35:04,696] A new study created in memory with name: no-name-9379b238-f26b-470c-a7bf-0525945a3f07
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2913457921197838 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([28, 1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incor

In [229]:
best_params

{'learning_rate': 0.0037751854616517506,
 'hidden_dim': 291,
 'num_layers': 2,
 'dropout': 0.08608821446980097}

In [238]:
best_lr = best_params['learning_rate']
best_hd = best_params['hidden_dim']
best_nl = best_params['num_layers']
best_do = best_params['dropout']

In [261]:
X_tv = np.concatenate((X_train, X_val), axis=0)
y_tv = np.concatenate((y_train, y_val), axis=0)

tv_dataset = CustomDataset(X_tv, y_tv)
tv_dataloader = DataLoader(tv_dataset, batch_size=batch_size, shuffle=False)

In [263]:
cv_model = GRUModel(input_dim, best_hd, best_nl, output_dim, best_do)

In [264]:
cv_num_epochs = 10
cv_criterion = nn.MSELoss()
cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=best_lr)

In [265]:
# Training loop
for epoch in range(cv_num_epochs):
    cv_model.train()
    for inputs, targets in tv_dataloader:
        cv_optimizer.zero_grad()
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        loss = cv_criterion(outputs, targets)
        loss.backward()
        cv_optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23, 1, 1])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [266]:
cv_model.eval()
with torch.no_grad():
    test_loss = 0
    for inputs, targets in test_dataloader:
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        test_loss += cv_criterion(outputs, targets).item()

    test_loss /= len(test_dataloader)

print(f'Test Loss: {test_loss}')

Test Loss: 8714271.818181818


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23, 1, 1])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [267]:
cv_model.eval()
with torch.no_grad():
    forecast_loss = 0
    for inputs, targets in forecast_dataloader:
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        forecast_loss += cv_criterion(outputs, targets).item()

    forecast_loss /= len(forecast_dataloader)

print(f'Forecast Loss: {forecast_loss}')

Forecast Loss: 13555126.666666666


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([22, 1, 1])) that is different to the input size (torch.Size([22, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
